**420-A58-SF - Algorithmes d'apprentissage non supervisé - Été 2021 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2021 Mikaël Swawola
<br/>
![Travaux Pratiques - Recherche de documents](static/02-02-A1-banner.png)
<br/>
**Objectif: Lors de l'exploration d'un jeu de données constitué de documents textes - tels que des pages Wikipedia, des articles de presse, StackOverflow, etc., il est courant de chercher à trouver quels sont les documents similaires. L'objectif de cet exercice est de mettre en oeuvre les techniques de recherche adaptées (ici les plus proches voisins) à ce type de données. Les documents utilisés sont les pages Wikipedia de personnalités.**

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

# Le reste des modules sera importé au fur et à mesure des exercices ...

L'archive `people.zip` contient 4 fichiers:

* **people_wiki.csv**: jeu de données consituté des pages Wikipedia de personnalités
* **people_wiki_map_index_to_word.json**: mapping entre les mots et les indices
* **people_wiki_word_count.npz**: vecteurs d'occurence des mots (word count / sacs de mot) pour chaque document
* **people_wiki_tf_idf.npz**: vecteurs TF-IDF pour chaque document

Dans l'énoncé de ce TP, les mots "article" et "document" sont interchangeables.

## 1 - Chargement du jeu de données

**Exercice 1-1 - À l'aide de la librairie Pandas, lire le fichier de données `people/people_wiki.csv`. Afin de permettre les opérations de type `join` effectuées plus loin dans le TP, nommez l'index de la trame de donnée `id`**

In [ ]:
# Compléter cette cellule ~ 2 lignes de code

In [ ]:
wiki = pd.read_csv('../../data/people/people_wiki.csv')
wiki.index.name = 'id'

**Exercice 1-2 - Afficher les 5 premières lignes de la trame de données. Quelles informations contiennent les colonnes ?**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code

In [ ]:
wiki.head()

## 2 - Extraction du nombre de mots

Les vecteurs d'occurence des mots (**word count**) du jeu de données ont été préalablement extrait dans le fichier `people/people_wiki_word_count.npz`. Ces vecteurs sont regroupés dans une matrice diluée (sparse), où la i-ème ligne donne le vecteur d'occurence des mots pour le i-ème document. Chaque colonne correspond à un mot unique apparaissant dans le jeu de données. Le mapping entre les mots et les indices est donné dans `people/people_wiki_map_index_to_word.json`

La fonction suivante permet le chargement des vecteurs d'occurence des mots:

In [ ]:
from scipy.sparse import csr_matrix

def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

La fonction ci-dessus utilise `csr_matrix` de la bibliothèque SciPy:<br/>
[class scipy.sparse.csr_matrix(arg1, shape=None, dtype=None, copy=False)](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)

**Exercice 2-1 - À l'aide de la fonction ci-dessus, charger le ficher contenant les vecteurs d'occurence des mots**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code

In [ ]:
from scipy.sparse import csr_matrix

word_count = load_sparse_csr('../../data/people/people_wiki_word_count.npz')

**Exercice 2-2 - En vous référant à la documentation de la fonction `csr_matrix`, convertissez la matrice `word_count` en tableau NumPy. Que constatez-vous ?**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code
word_count
59071*547979

In [ ]:
word_count.toarray()

**Exercice 2-3 - À l'aide du module json ou de la librairie Pandas, charger le ficher contenant le mapping entre les mots et les indices. Combien y a-t-il de mots dans le dictionnaire ?**

In [ ]:
# Compléter cette cellule ~ 2-3 lignes de code

In [ ]:
import json

with open('../../data/people/people_wiki_map_index_to_word.json') as f:
     map_index_to_word = json.load(f)
len(map_index_to_word)

**Exercice 2-4 (optionnel) - Extraire par vous-même les vecteurs d'occurence des mots. Un bon point de départ est la fonction `sklearn.CountVectorizer`**

In [ ]:
# Compléter cette cellule

## 3 - Recherche des plus proches voisins avec représentation word count

Commençons par trouver les voisins les plus proches de la page Wikipedia de **Barack Obama**. Les vecteurs d'occurence des mots (**word count**) seront utilisés pour représenter les articles et la **distance euclidienne** pour mesurer la similarité.

[class sklearn.neighbors.NearestNeighbors(*, n_neighbors=5, radius=1.0, algorithm='auto', leaf_size=30, metric='minkowski', p=2, metric_params=None, n_jobs=None)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html#sklearn.neighbors.NearestNeighbors)

**Exercice 3-1 - Quel est l'id correspondant à la page Wikipedia de barack Obama ?**

In [ ]:
# Compléter cette cellule ~ 1 ligne de code

In [ ]:
wiki['name'] == 'Barack Obama'
#wiki[wiki['name'] == 'Barack Obama']

**Exercice 3-2 - À l'aide de scikit-learn, rechercher les 10 pages Wikipedia de personnalités les plus similaires à la page de Barack Obama. Affichez les distances et noms de personalités dans une même trame de données**

In [ ]:
# Compléter cette cellule ~ 5-6 lignes de code

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric='euclidean', algorithm='brute').fit(word_count)
distances, indices = model.kneighbors(word_count[35817], n_neighbors=10)

In [ ]:
indices

In [ ]:
neighbors = pd.DataFrame({'distance':distances.flatten(), 'id':indices.flatten()}).set_index('id')
wiki.join(neighbors, on='id', how="right").sort_values(by='distance')[['name','distance']]

**Exercice 3-3 - Interprétez les résultats ci-dessus**

In [ ]:
# Compléter cette cellule

Les 10 personnalités sont toutes des politiciens, mais à peu près la moitié d'entre elles ont des liens assez ténus avec Obama, outre le fait qu'ils sont des politiciens.

* Francisco Barrio est un homme politique mexicain et ancien gouverneur de Chihuahua.
* Walter Mondale et Don Bonker sont des démocrates qui ont fait carrière à la fin des années 1970.
* Wynn Normington Hugh-Jones est un ancien diplomate britannique et fonctionnaire du Parti libéral.
* Andy Anstett est un ancien politicien au Manitoba, au Canada.

**Exercice 3-4 - Affichez les mots les plus fréquents des pages de Barack Obama et Francisco Barrio**

Afin de pouvoir reconnaître rapidement les mots d'une grande importance, la fonction suivante permettant d'obtenir la colonne `word_count` est fournie.

In [ ]:
def unpack_dict(matrix, map_index_to_word):
    table = sorted(map_index_to_word, key=map_index_to_word.get)
   
    data = matrix.data
    indices = matrix.indices
    indptr = matrix.indptr
    
    num_doc = matrix.shape[0]

    return [{k:v for k,v in zip([table[word_id] for word_id in indices[indptr[i]:indptr[i+1]] ],
                                 data[indptr[i]:indptr[i+1]].tolist())} for i in range(num_doc) ]

In [ ]:
# Compléter cette cellule ~ 2 lignes de code

In [ ]:
wiki['word_count'] = unpack_dict(word_count, map_index_to_word)
wiki['word_count']

**Exercice 3-5 - Créer une fonction `top_words`, permattant d'afficher les mots les plus fréquents d'une page donnée**

In [ ]:
# Compléter cette cellule ~ 10 lignes de code

In [ ]:
def top_words(name):
    """
    Retourne la table des mots les plus fréquents d'une page Wikipedia du jeu de données.
    """
    row = wiki[wiki['name'] == name]
    word_count_df = pd.DataFrame(row['word_count'].apply(pd.Series).stack(), columns=["count"]).droplevel(0)
    word_count_df.index.name = 'word'
    
    return word_count_df.sort_values(by='count', ascending=False)

In [ ]:
obama_words = top_words('Barack Obama')
barrio_words = top_words('Francisco Barrio')
combined_words = obama_words.join(barrio_words, on='word', how="inner", lsuffix='_obama', rsuffix='_barrio')
combined_words.head(10)

## 4 - Recherche des plus proches voisins avec représentation TF-IDF

**Exercice 4 - Répétez les étapes des exercices de la partie 3 en utilisant cette fois-ci la représentation TF-IDF. Comparez avec les résultats obtenu par la représentation word count**

In [ ]:
# Compléter cette cellule ~ 14-20 lignes de code

In [ ]:
# Chargement des représentations TF-IDF
tf_idf = load_sparse_csr('../../data/people/people_wiki_tf_idf.npz')

# Recherche des 10 plus proches voisins
model_tf_idf = NearestNeighbors(metric='euclidean', algorithm='brute').fit(tf_idf)
distances, indices = model_tf_idf.kneighbors(tf_idf[35817], n_neighbors=10)

# Préparation de la trame de données des résultats
neighbors = pd.DataFrame({'distance':distances.flatten(), 'id':indices.flatten()}).set_index('id')
wiki.join(neighbors, on='id', how='right').sort_values(by='distance')[['name','distance']]

In [ ]:
# Affichage des mots les plus significatifs des deux pages
wiki['tf_idf'] = unpack_dict(tf_idf, map_index_to_word)
def top_words_tf_idf(name):
    row = wiki[wiki['name'] == name]
    tf_idf_df = pd.DataFrame(row['tf_idf'].apply(pd.Series).stack(), columns=["weight"]).droplevel(0)
    tf_idf_df.index.name = 'word'
    return tf_idf_df.sort_values(by='weight', ascending=False)

obama_words = top_words_tf_idf('Barack Obama')
barrio_words = top_words_tf_idf('Francisco Barrio')
combined_words = obama_words.join(barrio_words, on='word', how="inner", lsuffix='_obama', rsuffix='_barrio')
combined_words.head(10)

### Fin de l'atelier 02-02-A1